# PLRデータ分析例

PLRに保存したデータの取得例です。

本環境は [Jupyter Notebook](https://jupyter.org/) の形で構成されており、[Pythonスクリプト](https://www.python.jp/)を用いて記述しています。

ライブラリは公式イメージ [jupyter/scipy-notebook](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/selecting.html#jupyter-scipy-notebook) のものがインストールされています。

# チャンネルの取得

処理したいデータが格納されたチャンネルを取得します。cogtask.meにより格納されたデータを処理したい場合は、 `cog-pds-log` となります。


> 実行状態が `[*]` のまま先に進まない場合は 中断(■) を押して一旦中断し、再度実行してみてください。

In [2]:
import asyncio
from plrfs.rpc_client import PLRFSClient

loop = asyncio.get_event_loop()

client = await PLRFSClient(loop).connect()

channels = await client.get_files([])
for ch in channels:
    print('Channel', ch['name'])
cogtask_me_chs = [c for c in channels if c['name'] == 'cog-pds-log']
assert len(cogtask_me_chs) > 0, 'cogtask.meのチャンネルが見つかりません。'
cogtask_me_ch = cogtask_me_chs[0]
cogtask_me_ch

Channel family
Channel coworkers
Channel friends
Channel 共有テスト
Channel Garmin Connect活動記録
Channel Cloze Test学習記録
Channel cog-pds-log


{'kind': 'folder', 'id': '20210406135701_CwTF', 'name': 'cog-pds-log'}

データの読み込みは `PLRFSClient`から実施できます。これは以下のような関数を持っています。

In [3]:
help(client)

Help on PLRFSClient in module plrfs.rpc_client object:

class PLRFSClient(builtins.object)
 |  PLRFSClient(loop)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, loop)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  async connect(self)
 |  
 |  async get_file(self, path)
 |  
 |  async get_files(self, path)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



ファイルシステムを模した構造になっていて、 `client.get_files` で指定したチャンネル、アイテムの配下のアイテム一覧を取得することができます。また、`client.get_file` で指定したチャンネル、アイテムの配下のデータを取得することができます。

以下では、最新20件のデータをPandas https://pandas.pydata.org/ の表に格納しています。

In [12]:
import json
import pandas as pd

timeline_item_data = []
# チャンネル内のアイテムを取得する
timeline_items = await client.get_files([cogtask_me_ch['id']])

# 最新20件のデータを取得する。
for item in timeline_items[-20:]:
    # アイテムのプロパティを取得する
    timeline_properties = await client.get_files([cogtask_me_ch['id'], item['id']])
    assert 'cnt' in [p['name'] for p in timeline_properties]
    assert 'begin' in [p['name'] for p in timeline_properties]
    assert 'cogPDSJSON' in [p['name'] for p in timeline_properties]
    
    # プロパティごとのデータを取得する
    summary = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cnt'][0]])
    begin = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'begin'][0]])
    cogPDSJSON = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSJSON'][0]])
    if 'cogPDSUser' in [p['name'] for p in timeline_properties]:
        cogPDSUser = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSUser'][0]])
    else:
        cogPDSUser = None
    timeline_item_data.append((summary['content'].decode('utf8'), begin['content'].decode('utf8'),
                               cogPDSUser['content'].decode('utf8') if cogPDSUser is not None else None, json.loads(cogPDSJSON['content'])))

df = pd.DataFrame(timeline_item_data, columns=['Summary', 'Time', 'User', 'Detail'])
df

,Summary,Time,User,Detail
0,*数字記憶タスク: 数字記憶タスクを 2 回実施しました。(平均点 3 点),2021-06-08T12:06:14.442Z,axFekHfsZnLiSKv1fkr0,{'meta': {'task': {'id': '5D8FANl25QgifqYqnETm...
1,*動画視聴ログ記録: 動画を視聴しました: 再生時間 0 秒,2021-06-08T15:16:56.048Z,D0wjUoFu99bz13OjlCUG,{'meta': {'task': {'id': 'Sb7lnzVw86yQwLxmXFFA...
2,*動画視聴ログ記録: 動画が一時停止されました: 再生時間 3 秒,2021-06-08T15:17:39.827Z,D0wjUoFu99bz13OjlCUG,{'meta': {'task': {'id': 'Sb7lnzVw86yQwLxmXFFA...
3,*動画視聴ログ記録: 動画を視聴しました: 再生時間 3 秒,2021-06-08T15:17:41.608Z,D0wjUoFu99bz13OjlCUG,{'meta': {'task': {'id': 'Sb7lnzVw86yQwLxmXFFA...
4,Garminデータが同期されました: sleeps,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'sleeps', 'pulled': 1623..."
5,Garminデータが同期されました: dailies,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'dailies', 'pulled': 162..."
6,Garminデータが同期されました: stressDetails,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'stressDetails', 'pulled..."
7,Garminデータが同期されました: stressDetails,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'stressDetails', 'pulled..."
8,Garminデータが同期されました: epochs,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'epochs', 'pulled': 1623..."
9,Garminデータが同期されました: sleeps,2021-06-08T21:32:01.739Z,None,"{'meta': {'dataType': 'sleeps', 'pulled': 1623..."


各レコードの詳細データの形式は、 https://cogtask.me で実施したタスクの種類(動画視聴、数字記憶等)で異なります。試しに、最後に格納されたデータの内容を見てみましょう。

> 以下のセルは、Personaryにデータが1つも格納されていないと失敗します。

In [13]:
df['Detail'].values[-1]

{'meta': {'task': {'id': 'GARMIN-V1', 'title': 'Garmin Connect'},
  'garmin': {'dataType': 'epochs',
   'pulled': 1623190922469,
   'ping': 'qQLmByQ6vhXmRxgFierP'}},
 'data': [{'summaryId': 'x2cbe65b-60bfddd4-6',
   'activityType': 'WALKING',
   'activeKilocalories': 0,
   'steps': 9,
   'distanceInMeters': 7.04,
   'durationInSeconds': 900,
   'activeTimeInSeconds': 60,
   'startTimeInSeconds': 1623186900,
   'startTimeOffsetInSeconds': 32400,
   'met': 1,
   'intensity': 'ACTIVE',
   'meanMotionIntensity': 1,
   'maxMotionIntensity': 1},
  {'summaryId': 'x2cbe65b-60bfddd4-8',
   'activityType': 'SEDENTARY',
   'activeKilocalories': 2,
   'steps': 0,
   'distanceInMeters': 0,
   'durationInSeconds': 900,
   'activeTimeInSeconds': 840,
   'startTimeInSeconds': 1623186900,
   'startTimeOffsetInSeconds': 32400,
   'met': 1.0967885,
   'intensity': 'SEDENTARY',
   'meanMotionIntensity': 0,
   'maxMotionIntensity': 5},
  {'summaryId': 'x2cbe65b-60bfe158-6',
   'activityType': 'WALKING',
  

各詳細データには `meta` というプロパティがあり、そのデータの種類などの情報が格納されています。

In [14]:
df['Detail'].values[-1]['meta']

{'task': {'id': 'GARMIN-V1', 'title': 'Garmin Connect'},
 'garmin': {'dataType': 'epochs',
  'pulled': 1623190922469,
  'ping': 'qQLmByQ6vhXmRxgFierP'}}

# Garminデータの取得

試しに、チャンネルからGarminデータを取り出してみましょう。

In [17]:
timeline_item_data = []
# チャンネル内のアイテムを取得する
timeline_items = await client.get_files([cogtask_me_ch['id']])

for item in timeline_items:
    print('Processing...', item['id'])
    # アイテムのプロパティを取得する
    timeline_properties = await client.get_files([cogtask_me_ch['id'], item['id']])
    if 'cogPDSJSON' not in [p['name'] for p in timeline_properties]:
        continue
    
    # プロパティごとのデータを取得する
    cogPDSJSON = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSJSON'][0]])
    cogPDSJSONDict = json.loads(cogPDSJSON['content'])
    if 'meta' in cogPDSJSONDict and 'task' in cogPDSJSONDict['meta'] and cogPDSJSONDict['meta']['task']['id'] == 'GARMIN-V1':
        assert 'cnt' in [p['name'] for p in timeline_properties]
        assert 'begin' in [p['name'] for p in timeline_properties]
        summary = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cnt'][0]])
        begin = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'begin'][0]])
        timeline_item_data.append((summary['content'].decode('utf8'), begin['content'].decode('utf8'), cogPDSJSONDict))

df = pd.DataFrame(timeline_item_data, columns=['Summary', 'Time', 'Detail'])
df

Processing... #20210406224204_dTJy
Processing... #20210406224206_QA95
Processing... #20210406230009_fhsd
Processing... #20210406230003_EknP
Processing... #20210406225911_WCmC
Processing... #20210406230006_8yVF
Processing... #20210407010607_FEja
Processing... #20210407010804_hdRo
Processing... #20210407011104_UzZ5
Processing... #20210407011404_jbSi
Processing... #20210407012509_e5HS
Processing... #20210407012604_uFFV
Processing... #20210520010909_Cq8d
Processing... #20210520010905_mr9s
Processing... #20210407135905_HQPh
Processing... #20210408032308_Sw7p
Processing... #20210408032304_oUp5
Processing... #20210422020703_RGI1
Processing... #20210429213205_HhVe
Processing... #20210510074103_yDbD
Processing... #20210511023003_wAqO
Processing... #20210513051402_P0ae
Processing... #20210513052009_ED-2
Processing... #20210513052003_3ml7
Processing... #20210513052908_E1k5
Processing... #20210513052903_cU1t
Processing... #20210513134109_d4uT
Processing... #20210513134015_oCiE
Processing... #20210

,Summary,Time,Detail
0,Garminデータが同期されました: sleeps,2021-06-08T22:22:02.469Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
1,Garminデータが同期されました: stressDetails,2021-06-08T22:22:02.469Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
2,Garminデータが同期されました: dailies,2021-06-08T22:22:02.469Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
3,Garminデータが同期されました: epochs,2021-06-08T22:22:02.469Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
4,Garminデータが同期されました: stressDetails,2021-06-08T22:37:02.585Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
5,Garminデータが同期されました: epochs,2021-06-08T22:37:02.585Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."
6,Garminデータが同期されました: dailies,2021-06-08T22:37:02.585Z,"{'meta': {'task': {'id': 'GARMIN-V1', 'title':..."


このようにして、Personary上のデータをメモリに読み込むことができます。これらの内容を**暗号化等をかけていないファイル等に出力する際は十分に取り扱いに注意**してください。